# Base on pygame

In [1]:
# The following code is generated by ChatGPT4
import pygame
import random
import heapq
pygame.init()

# Window setup
width, height = 400, 400
win = pygame.display.set_mode((width, height))
pygame.display.set_caption("Maze with A* Agents and Trash Collection")

clock = pygame.time.Clock()

# Maze parameters
cols, rows = 10, 10
cell_size = 40
maze_width = cols * cell_size
maze_height = rows * cell_size

# Define the goal cell coordinates
goal_cell = (cols - 1, rows - 1)  # Bottom-right corner

# Initialize the maze grid
maze = [[{'visited': False, 'walls': {'top': True, 'right': True, 'bottom': True, 'left': True}} for _ in range(cols)] for _ in range(rows)]

def generate_maze(x, y):
    maze[y][x]['visited'] = True
    directions = ['top', 'right', 'bottom', 'left']
    random.shuffle(directions)
    for direction in directions:
        nx, ny = x, y
        if direction == 'top':
            ny -= 1
        elif direction == 'right':
            nx += 1
        elif direction == 'bottom':
            ny += 1
        elif direction == 'left':
            nx -= 1
        if 0 <= nx < cols and 0 <= ny < rows and not maze[ny][nx]['visited']:
            # Remove walls between current cell and next cell
            maze[y][x]['walls'][direction] = False
            opposite = {'top': 'bottom', 'right': 'left', 'bottom': 'top', 'left': 'right'}
            maze[ny][nx]['walls'][opposite[direction]] = False
            generate_maze(nx, ny)

# Generate the maze starting from the top-left corner
generate_maze(0, 0)

def draw_maze():
    for y in range(rows):
        for x in range(cols):
            cell = maze[y][x]
            x_pos = x * cell_size
            y_pos = y * cell_size
            if cell['walls']['top']:
                pygame.draw.line(win, (255, 255, 255), (x_pos, y_pos), (x_pos + cell_size, y_pos), 2)
            if cell['walls']['right']:
                pygame.draw.line(win, (255, 255, 255), (x_pos + cell_size, y_pos), (x_pos + cell_size, y_pos + cell_size), 2)
            if cell['walls']['bottom']:
                pygame.draw.line(win, (255, 255, 255), (x_pos + cell_size, y_pos + cell_size), (x_pos, y_pos + cell_size), 2)
            if cell['walls']['left']:
                pygame.draw.line(win, (255, 255, 255), (x_pos, y_pos + cell_size), (x_pos, y_pos), 2)

def heuristic(a, b):
    # Manhattan distance on a square grid
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def get_neighbors(maze, cell):
    x, y = cell
    neighbors = []
    directions = {'top': (0, -1), 'right': (1, 0), 'bottom': (0, 1), 'left': (-1, 0)}
    for direction, (dx, dy) in directions.items():
        nx, ny = x + dx, y + dy
        if 0 <= nx < cols and 0 <= ny < rows:
            # Check if there is no wall between current cell and neighbor
            if not maze[y][x]['walls'][direction]:
                neighbors.append((nx, ny))
    return neighbors

def a_star_search(maze, start, goal):
    # Create a priority queue and hash sets for visited nodes
    queue = []
    heapq.heappush(queue, (0, start))
    
    came_from = {}  # For path reconstruction
    cost_so_far = {}
    came_from[start] = None
    cost_so_far[start] = 0
    
    while queue:
        _, current = heapq.heappop(queue)
        
        if current == goal:
            break
        
        neighbors = get_neighbors(maze, current)
        for next_cell in neighbors:
            new_cost = cost_so_far[current] + 1  # Assuming all moves cost 1
            if next_cell not in cost_so_far or new_cost < cost_so_far[next_cell]:
                cost_so_far[next_cell] = new_cost
                priority = new_cost + heuristic(goal, next_cell)
                heapq.heappush(queue, (priority, next_cell))
                came_from[next_cell] = current
                
    # Reconstruct path
    current = goal
    path = []
    while current != start:
        path.append(current)
        current = came_from.get(current)
        if current is None:
            return []  # No path found
    path.append(start)
    path.reverse()
    return path

# Number of trash items
num_trash_items = 5
trash_items = []

# Function to place trash items in random accessible cells
def place_trash_items():
    placed = 0
    while placed < num_trash_items:
        x = random.randint(0, cols - 1)
        y = random.randint(0, rows - 1)
        if (x, y) != goal_cell and not all(maze[y][x]['walls'].values()) and (x, y) not in trash_items:
            trash_items.append((x, y))
            placed += 1

place_trash_items()

class Agent:
    def __init__(self, x, y, color=(0, 255, 0)):
        self.cell_x = x  # Cell coordinates
        self.cell_y = y
        self.x = x * cell_size + (cell_size - 20) // 2  # Pixel coordinates
        self.y = y * cell_size + (cell_size - 20) // 2
        self.color = color
        self.size = 20
        self.speed = 2
        self.path = []
        self.path_index = 0
        self.calculate_path()
    
    def calculate_path(self):
        start = (self.cell_x, self.cell_y)
        if trash_items:
            # Find the nearest trash item
            nearest_trash = min(trash_items, key=lambda t: heuristic(start, t))
            self.target = nearest_trash
        else:
            # No trash items left, head to goal
            self.target = goal_cell
        self.path = a_star_search(maze, start, self.target)
        if len(self.path) > 1:
            self.path_index = 1
        else:
            self.path_index = 0
    
    def move(self):
        if self.path_index < len(self.path):
            target_cell = self.path[self.path_index]
            target_x = target_cell[0] * cell_size + (cell_size - self.size) // 2
            target_y = target_cell[1] * cell_size + (cell_size - self.size) // 2
            
            # Move towards target
            dx = target_x - self.x
            dy = target_y - self.y
            dist = (dx ** 2 + dy ** 2) ** 0.5
            if dist != 0:
                dx /= dist
                dy /= dist
                self.x += dx * self.speed
                self.y += dy * self.speed
                
                # Check if reached the target cell
                if abs(self.x - target_x) < self.speed and abs(self.y - target_y) < self.speed:
                    self.x = target_x
                    self.y = target_y
                    self.cell_x, self.cell_y = target_cell
                    self.path_index += 1
        else:
            # Reached the end of path
            if self.target in trash_items:
                # Collect the trash
                trash_items.remove(self.target)
                # Recalculate path to next trash item or goal
                self.calculate_path()
            elif self.target == goal_cell:
                # Reached the goal
                pass  # Agent has completed its task
            else:
                # Target no longer valid (e.g., trash collected by another agent)
                # Recalculate path to next trash item or goal
                self.calculate_path()
    
    def draw(self, win):
        pygame.draw.rect(win, self.color, (self.x, self.y, self.size, self.size))

# Number of agents
num_agents = 5
agents = []

# Function to find a random free cell for agent starting positions
def get_random_free_cell():
    while True:
        x = random.randint(0, cols - 1)
        y = random.randint(0, rows - 1)
        if (x, y) != goal_cell and (x, y) not in trash_items and not all(maze[y][x]['walls'].values()):
            return x, y

for _ in range(num_agents):
    agent_cell_x, agent_cell_y = get_random_free_cell()
    agents.append(Agent(agent_cell_x, agent_cell_y))

run = True

while run:
    clock.tick(30)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            run = False

    # Update agents
    for agent in agents:
        agent.move()

    # Clear the screen
    win.fill((0, 0, 0))

    # Draw the maze
    draw_maze()

    # Draw trash items
    for trash in trash_items:
        trash_x = trash[0] * cell_size + cell_size // 2
        trash_y = trash[1] * cell_size + cell_size // 2
        pygame.draw.circle(win, (200, 0, 200), (trash_x, trash_y), cell_size // 4)

    # Draw agents
    for agent in agents:
        agent.draw(win)

    # Draw the goal
    goal_x = goal_cell[0] * cell_size + cell_size // 2
    goal_y = goal_cell[1] * cell_size + cell_size // 2
    pygame.draw.circle(win, (255, 215, 0), (goal_x, goal_y), cell_size // 4)

    pygame.display.update()

pygame.quit()

pygame 2.6.1 (SDL 2.28.4, Python 3.11.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


KeyboardInterrupt: 